In [12]:
#Importing Dependencies

import pandas as pd
import requests
import json
import time
from datetime import datetime
import pickle

In [61]:
import great_expectations as ge

In [70]:
from great_expectations import Dataset

ImportError: cannot import name 'Dataset' from 'great_expectations' (C:\Users\imarr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\great_expectations\__init__.py)

In [8]:
#Extracting the data from the api

APP_ID = 'e2e4d023'  # Replace with actual API endpoint
API_KEY = API_KEY = 'bf7baff0c295d4fc1ab3e121837d3d68'
POLL_INTERVAL = 60  # Time in seconds between requests

# Define the endpoint and parameters for scheduled data
station_code = 'WAT'  # Example station code (London Waterloo)
url = f'https://transportapi.com/v3/uk/train/station/{station_code}/live.json'

params = {
    'app_id': APP_ID,
    'app_key': API_KEY,
    #'time_of_day': '19:00',
    #'request_time': '2024-10-30T18:50:00+00:00',
    'darwin': 'false',  
    'train_status': 'passenger',  # Status filter, e.g., passenger trains only
    'live' :'True'
}
response = requests.get(url, params=params)

if response.status_code == 200:
    response= response.json()
else:
    print(f"Failed to fetch data: {response.status_code}")

In [9]:
response

{'date': '2024-11-01',
 'time_of_day': '13:39',
 'request_time': '2024-11-01T13:39:48+00:00',
 'station_name': 'London Waterloo',
 'station_code': 'WAT',
 'departures': {'all': [{'mode': 'train',
    'service': '24629204',
    'train_uid': 'L60674',
    'platform': '8',
    'operator': 'SW',
    'operator_name': 'South Western Railway',
    'aimed_departure_time': '13:42',
    'aimed_arrival_time': None,
    'aimed_pass_time': None,
    'origin_name': 'London Waterloo',
    'destination_name': 'Basingstoke',
    'source': 'Network Rail',
    'category': 'OO',
    'service_timetable': {'id': 'https://transportapi.com/v3/uk/train/service_timetables/L60674:2024-11-01.json?app_id=e2e4d023&app_key=bf7baff0c295d4fc1ab3e121837d3d68&live=true'},
    'status': 'STARTS HERE',
    'expected_arrival_time': None,
    'expected_departure_time': '13:42',
    'best_arrival_estimate_mins': None,
    'best_departure_estimate_mins': 2},
   {'mode': 'train',
    'service': '24673105',
    'train_uid': 'L5

In [10]:
with open("response.pkl", "wb") as f:
    pickle.dump(response, f)

In [11]:
train_columns = []

date = response['date']
time_of_day = response['time_of_day']
station_name = response['station_name']

train_info = {
    'date': date,
    'time_of_day': time_of_day,
    'station_name': station_name
}
train_columns.append(train_info)

train_columns

[{'date': '2024-11-01',
  'time_of_day': '13:39',
  'station_name': 'London Waterloo'}]

In [13]:
train_columns_df = pd.DataFrame(train_columns)
train_columns_df.reset_index(inplace=True)
train_columns_df

,index,date,time_of_day,station_name
0,0,2024-11-01,13:39,London Waterloo


In [19]:
train__departure_columns = []

for columns in response['departures']['all']:
    try:
        row ={
            'mode':columns['mode'],
            'train_uid': columns['train_uid'],
            'origin_name': columns['origin_name'],
            'operator_name':columns['operator_name'],
            'platform': columns['platform'],
            'destination_name': columns['destination_name'],
            'aimed_departure_time': columns['aimed_departure_time'],
            'expected_departure_time': columns['expected_departure_time'],
            'best_departure_estimate_mins': columns['best_departure_estimate_mins'],
            'aimed_arrival_time': columns['aimed_arrival_time']
        }
        train__departure_columns.append( row)
    except (KeyError, TypeError) as e:
        print(f"Error processing columns: {e}")     

print(train__departure_columns)

[{'mode': 'train', 'train_uid': 'L60674', 'origin_name': 'London Waterloo', 'operator_name': 'South Western Railway', 'platform': '8', 'destination_name': 'Basingstoke', 'aimed_departure_time': '13:42', 'expected_departure_time': '13:42', 'best_departure_estimate_mins': 2, 'aimed_arrival_time': None}, {'mode': 'train', 'train_uid': 'L59931', 'origin_name': 'London Waterloo', 'operator_name': 'South Western Railway', 'platform': '5', 'destination_name': 'Shepperton', 'aimed_departure_time': '13:42', 'expected_departure_time': '13:42', 'best_departure_estimate_mins': 2, 'aimed_arrival_time': None}, {'mode': 'train', 'train_uid': 'L61473', 'origin_name': 'London Waterloo', 'operator_name': 'South Western Railway', 'platform': '9', 'destination_name': 'Haslemere', 'aimed_departure_time': '13:45', 'expected_departure_time': '13:45', 'best_departure_estimate_mins': 5, 'aimed_arrival_time': None}, {'mode': 'train', 'train_uid': 'L60966', 'origin_name': 'London Waterloo', 'operator_name': 'Sou

In [20]:
train__departure_columns_df = pd.DataFrame(train__departure_columns)
train__departure_columns_df.reset_index(inplace=True)
train__departure_columns_df

,index,mode,train_uid,origin_name,operator_name,platform,destination_name,aimed_departure_time,expected_departure_time,best_departure_estimate_mins,aimed_arrival_time
0,0,train,L60674,London Waterloo,South Western Railway,8,Basingstoke,13:42,13:42,2,None
1,1,train,L59931,London Waterloo,South Western Railway,5,Shepperton,13:42,13:42,2,None
2,2,train,L61473,London Waterloo,South Western Railway,9,Haslemere,13:45,13:45,5,None
3,3,train,L60966,London Waterloo,South Western Railway,4,Chessington South,13:47,13:47,7,None
4,4,train,L58580,London Waterloo,South Western Railway,23,Reading,13:50,13:50,10,None
5,5,train,L57250,London Waterloo,South Western Railway,7,Salisbury,13:50,13:50,10,None
6,6,train,L59330,London Waterloo,South Western Railway,3,Woking,13:50,13:50,10,None
7,7,train,L61787,London Waterloo,South Western Railway,24,Weybridge,13:52,13:52,12,None
8,8,train,Y00066,London Waterloo,South Western Railway,10,Alton,13:53,13:53,13,None
9,9,train,L58886,London Waterloo,South Western Railway,1,Guildford,13:54,13:54,14,None


In [53]:
#Merging both dataframes

train_schedule_df = pd.merge(train_columns_df, train__departure_columns_df, on='index', how='outer' )
train_schedule_df

,index,date,time_of_day,station_name,mode,train_uid,origin_name,operator_name,platform,destination_name,aimed_departure_time,expected_departure_time,best_departure_estimate_mins,aimed_arrival_time
0,0,2024-11-01,13:39,London Waterloo,train,L60674,London Waterloo,South Western Railway,8,Basingstoke,13:42,13:42,2,None
1,1,NaN,NaN,NaN,train,L59931,London Waterloo,South Western Railway,5,Shepperton,13:42,13:42,2,None
2,2,NaN,NaN,NaN,train,L61473,London Waterloo,South Western Railway,9,Haslemere,13:45,13:45,5,None
3,3,NaN,NaN,NaN,train,L60966,London Waterloo,South Western Railway,4,Chessington South,13:47,13:47,7,None
4,4,NaN,NaN,NaN,train,L58580,London Waterloo,South Western Railway,23,Reading,13:50,13:50,10,None
5,5,NaN,NaN,NaN,train,L57250,London Waterloo,South Western Railway,7,Salisbury,13:50,13:50,10,None
6,6,NaN,NaN,NaN,train,L59330,London Waterloo,South Western Railway,3,Woking,13:50,13:50,10,None
7,7,NaN,NaN,NaN,train,L61787,London Waterloo,South Western Railway,24,Weybridge,13:52,13:52,12,None
8,8,NaN,NaN,NaN,train,Y00066,London Waterloo,South Western Railway,10,Alton,13:53,13:53,13,None
9,9,NaN,NaN,NaN,train,L58886,London Waterloo,South Western Railway,1,Guildford,13:54,13:54,14,None


In [54]:
#Renaming the time of day column

train_schedule_df.rename(columns= {'time_of_day':'schedule_time'}, inplace = True)

In [62]:
train_schedule_df.rename(columns= {'index':'id'}, inplace = True)

In [63]:
train_schedule_df

,id,date,schedule_time,station_name,mode,train_uid,origin_name,operator_name,platform,destination_name,aimed_departure_time,expected_departure_time,best_departure_estimate_mins,aimed_arrival_time
0,0,2024-11-01,13:39,London Waterloo,train,L60674,London Waterloo,South Western Railway,8,Basingstoke,13:42,13:42,2,Unknonwn
1,1,2024-11-01,13:39,London Waterloo,train,L59931,London Waterloo,South Western Railway,5,Shepperton,13:42,13:42,2,Unknonwn
2,2,2024-11-01,13:39,London Waterloo,train,L61473,London Waterloo,South Western Railway,9,Haslemere,13:45,13:45,5,Unknonwn
3,3,2024-11-01,13:39,London Waterloo,train,L60966,London Waterloo,South Western Railway,4,Chessington South,13:47,13:47,7,Unknonwn
4,4,2024-11-01,13:39,London Waterloo,train,L58580,London Waterloo,South Western Railway,23,Reading,13:50,13:50,10,Unknonwn
5,5,2024-11-01,13:39,London Waterloo,train,L57250,London Waterloo,South Western Railway,7,Salisbury,13:50,13:50,10,Unknonwn
6,6,2024-11-01,13:39,London Waterloo,train,L59330,London Waterloo,South Western Railway,3,Woking,13:50,13:50,10,Unknonwn
7,7,2024-11-01,13:39,London Waterloo,train,L61787,London Waterloo,South Western Railway,24,Weybridge,13:52,13:52,12,Unknonwn
8,8,2024-11-01,13:39,London Waterloo,train,Y00066,London Waterloo,South Western Railway,10,Alton,13:53,13:53,13,Unknonwn
9,9,2024-11-01,13:39,London Waterloo,train,L58886,London Waterloo,South Western Railway,1,Guildford,13:54,13:54,14,Unknonwn


In [56]:
#Filling up missing values

train_schedule_df['date'].fillna('2024-11-01', inplace=True)
train_schedule_df.reset_index(drop=True, inplace=True)
train_schedule_df = train_schedule_df.loc[:, ~train_schedule_df.columns.str.contains('^level_')]

train_schedule_df['schedule_time'].fillna('13:39', inplace=True)
train_schedule_df.reset_index(drop=True, inplace=True)
train_schedule_df = train_schedule_df.loc[:, ~train_schedule_df.columns.str.contains('^level_')]

train_schedule_df['station_name'].fillna('London Waterloo', inplace=True)
train_schedule_df.reset_index(drop=True, inplace=True)
train_schedule_df = train_schedule_df.loc[:, ~train_schedule_df.columns.str.contains('^level_')]

train_schedule_df['aimed_arrival_time'].fillna('Unknonwn', inplace=True)
train_schedule_df.reset_index(drop=True, inplace=True)
train_schedule_df = train_schedule_df.loc[:, ~train_schedule_df.columns.str.contains('^level_')]



In [64]:
train_schedule_df

,id,date,schedule_time,station_name,mode,train_uid,origin_name,operator_name,platform,destination_name,aimed_departure_time,expected_departure_time,best_departure_estimate_mins,aimed_arrival_time
0,0,2024-11-01,13:39,London Waterloo,train,L60674,London Waterloo,South Western Railway,8,Basingstoke,13:42,13:42,2,Unknonwn
1,1,2024-11-01,13:39,London Waterloo,train,L59931,London Waterloo,South Western Railway,5,Shepperton,13:42,13:42,2,Unknonwn
2,2,2024-11-01,13:39,London Waterloo,train,L61473,London Waterloo,South Western Railway,9,Haslemere,13:45,13:45,5,Unknonwn
3,3,2024-11-01,13:39,London Waterloo,train,L60966,London Waterloo,South Western Railway,4,Chessington South,13:47,13:47,7,Unknonwn
4,4,2024-11-01,13:39,London Waterloo,train,L58580,London Waterloo,South Western Railway,23,Reading,13:50,13:50,10,Unknonwn
5,5,2024-11-01,13:39,London Waterloo,train,L57250,London Waterloo,South Western Railway,7,Salisbury,13:50,13:50,10,Unknonwn
6,6,2024-11-01,13:39,London Waterloo,train,L59330,London Waterloo,South Western Railway,3,Woking,13:50,13:50,10,Unknonwn
7,7,2024-11-01,13:39,London Waterloo,train,L61787,London Waterloo,South Western Railway,24,Weybridge,13:52,13:52,12,Unknonwn
8,8,2024-11-01,13:39,London Waterloo,train,Y00066,London Waterloo,South Western Railway,10,Alton,13:53,13:53,13,Unknonwn
9,9,2024-11-01,13:39,London Waterloo,train,L58886,London Waterloo,South Western Railway,1,Guildford,13:54,13:54,14,Unknonwn


In [65]:
train_schedule_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   id                            50 non-null     int64 
 1   date                          50 non-null     object
 2   schedule_time                 50 non-null     object
 3   station_name                  50 non-null     object
 4   mode                          50 non-null     object
 5   train_uid                     50 non-null     object
 6   origin_name                   50 non-null     object
 7   operator_name                 50 non-null     object
 8   platform                      50 non-null     object
 9   destination_name              50 non-null     object
 10  aimed_departure_time          50 non-null     object
 11  expected_departure_time       50 non-null     object
 12  best_departure_estimate_mins  50 non-null     int64 
 13  aimed_arrival_time    

In [66]:
#Saving file to CSV
train_schedule_df.to_csv('train_schedule.csv')

In [79]:
print(ge.__version__)

1.2.0


In [80]:
pip install -U great_expectations


   ---------------------------------------- 0.0/5.0 MB ? eta -:--:--
   ------------ --------------------------- 1.6/5.0 MB 4.1 MB/s eta 0:00:01
   ------------------------------ --------- 3.9/5.0 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------  5.0/5.0 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------  5.0/5.0 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------  5.0/5.0 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------- 5.0/5.0 MB 2.7 MB/s eta 0:00:00
  Attempting uninstall: great_expectations
    Found existing installation: great-expectations 1.2.0
    Uninstalling great-expectations-1.2.0:
      Successfully uninstalled great-expectations-1.2.0
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\imarr\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [88]:
pip uninstall great_expectations

^C
Note: you may need to restart the kernel to use updated packages.


In [89]:
pip install great_expectations

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\imarr\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
pip install great_expectations

  Obtaining dependency information for great_expectations from https://files.pythonhosted.org/packages/08/65/c23361666868ad2a1ae4192d6be133717d971dda6dcb3d1472ddb0e2dd82/great_expectations-1.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for altair<5.0.0,>=4.2.1 from https://files.pythonhosted.org/packages/18/62/47452306e84d4d2e67f9c559380aeb230f5e6ca84fafb428dd36b96a99ba/altair-4.2.2-py3-none-any.whl.metadata
  Obtaining dependency information for cryptography>=3.2 from https://files.pythonhosted.org/packages/87/5c/3dab83cc4aba1f4b0e733e3f0c3e7d4386440d660ba5b1e3ff995feb734d/cryptography-43.0.3-cp39-abi3-win_amd64.whl.metadata
  Obtaining dependency information for jinja2>=2.10 from https://files.pythonhosted.org/packages/31/80/3a54838c3fb461f6fec263ebf3a3a41771bd05190238de3486aae8540c36/jinja2-3.1.4-py3-none-any.whl.metadata
  Obtaining dependency information for jsonschema>=2.5.1 from https://files.pythonhosted.org/packages/69/4a/4f9dbeb84e8850557c02365a0eee0649abe


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import great_expectations as ge

In [ ]:
pip show great_expectations


Name: great-expectations
Version: 1.2.1
Summary: Always know what to expect from your data.
Home-page: https://greatexpectations.io
Author: The Great Expectations Team
Author-email: team@greatexpectations.io
License: Apache-2.0
Location: c:\Users\imarr\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: altair, cryptography, jinja2, jsonschema, marshmallow, mistune, numpy, numpy, packaging, pandas, pandas, posthog, pydantic, pyparsing, python-dateutil, requests, ruamel.yaml, scipy, tqdm, typing-extensions, tzlocal
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [1]:

from great_expectations.data_context import DataContext
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.validator.validator import Validator

ImportError: cannot import name 'DataContext' from 'great_expectations.data_context' (c:\Users\imarr\AppData\Local\Programs\Python\Python312\Lib\site-packages\great_expectations\data_context\__init__.py)

In [77]:

# Initialize a DataContext
context = ge.get_context()

# Assuming train_schedule_df is your DataFrame
ge_df = train_schedule_df  # replace with your actual DataFrame

# Set up the RuntimeBatchRequest to pass the DataFrame to the Validator
batch_request = RuntimeBatchRequest(
    datasource_name="my_pandas_datasource",
    data_connector_name="default_runtime_data_connector_name",
    data_asset_name="train_schedule_data",  # name the data asset
    runtime_parameters={"batch_data": train_schedule_df},
    batch_identifiers={"default_identifier_name": "batch1"}
)

# Create a Validator directly from the context and batch request
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name="my_suite"  # Create or use an existing expectation suite name
)

# Apply expectations directly
validator.expect_column_values_to_not_be_null("date")
validator.expect_column_values_to_be_of_type("station_name", "str")

# Run validation and print results
results = validator.validate()
print(results)

DataContextError: ExpectationSuite with name my_suite was not found.

In [68]:
train_schedule_ge_df= ge.dataset.PandasDataset(train_schedule_df)

AttributeError: module 'great_expectations' has no attribute 'dataset'